In [1]:
# these packages can be installed from https://github.com/hengenlab

import neuraltoolkit as ntk
import musclebeachtools as mbt
import numpy as np
import os
import mrestimator as mre   # pip install
import pandas as pd
import scipy as sc

from datetime import date
today = date.today()
today = today.strftime("%Y_%m_%d")

INFO     Loaded mrestimator v0.1.8, writing to /tmp/mre_xu.yifan/
INFO     Numba not available, skipping parallelization


In [3]:
def Calc_BR(A_t, k_max,ava_binsz,pltname):

    dt = ava_binsz*1000
    kmax = k_max

    src = mre. input_handler(A_t)
    rks = mre. coefficients(src , steps =(1, kmax) , dt=dt, dtunit='ms', method='trialseparated')

    fit1 = mre.fit(rks , fitfunc='complex')
    fit2 = mre.fit(rks , fitfunc='exp_offset')

    fig,ax1 = plt.subplots()
    ax1.plot(rks.steps, rks.coefficients, '.k', alpha = 0.2, label=r'Data')

    ax1.plot(rks.steps, mre.f_complex(rks.steps*dt, *fit1.popt), label='complex m={:.5f}'.format(fit1.mre))
    ax1.plot(rks.steps, mre.f_exponential_offset(rks.steps*dt, *fit2.popt), label='exp + offset m={:.5f}'.format(fit2.mre))

    ax1.set_xlabel(r'Time lag $\delta t$')
    ax1.set_ylabel(r'Autocorrelation $r_{\delta t}$')
    ax1.legend()
    plt.savefig(pltname)
    plt.close()

    fit_acc1 = sc.stats.pearsonr(rks.coefficients, mre.f_complex(rks.steps*dt, *fit1.popt))[0]
    fit_acc2 = sc.stats.pearsonr(rks.coefficients, mre.f_exponential_offset(rks.steps*dt, *fit2.popt))[0]

    return fit1.mre, fit2.mre, fit_acc1, fit_acc2

In [ ]:
neurons = np.load('/media/HlabShare/Yifan_Work/Document/spk_data/XYF12/block1/XYF12_b1_neurons_quality.npy',allow_pickle = True)

goodCells = [neuron for neuron in neurons if neuron.quality in [1,2]]

for i in np.arange(0,len(goodCells)):
    goodCells[i].remove_large_amplitude_spikes(3.5,lstd_deviation=True, start=False, end=False, lplot=False)

goodCells = [neuron for neuron in goodCells if neuron.cell_type == 'RSU']

nrn_time = 1          # total hours we want to analyze
ava_binsz = 0.004     # 4ms binsize
data= mbt.n_spiketimes_to_spikewords(goodCells,ava_binsz,0,3600*nrn_time,0)    #1 is for binary, 0 is for count actual number

A_t = np.sum(data,0)

calcbin = 120   # get branching ratio every 2 min

thistime = int(np.floor(A_t.shape[0]/(calcbin/ava_binsz)))
minbin = int(calcbin/ava_binsz)

k_max = 500
BR_list_all = []

for i in np.arange(thistime):

    pltname = "XYF12_2min_num" + str(i)
    BR_list = np.asarray(Calc_BR(A_t[i*minbin:(i+1)*minbin], k_max, ava_binsz, pltname))
    BR_list_all.append(BR_list)